In [1]:
# Bring in necessary libraries
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import datetime

# Create a connection to the database
conn = sqlite3.connect('database.db')
# Create a cursor object
cur = conn.cursor()
# Read in the 3 csv files that represent the 3 tables for our country club database
df_book = pd.read_csv(r'C:\Users\ryanm\Desktop\Springboard\Mod19 - CountryClub SQL Exercise\Bookings.csv')
df_mem = pd.read_csv(r'C:\Users\ryanm\Desktop\Springboard\Mod19 - CountryClub SQL Exercise\Members.csv')
df_fac = pd.read_csv(r'C:\Users\ryanm\Desktop\Springboard\Mod19 - CountryClub SQL Exercise\Facilities.csv')
# Convert the csv files to sql
df_book.to_sql('Bookings', conn, if_exists='replace', index = False)
df_mem.to_sql('Members', conn, if_exists='replace', index = False)
df_fac.to_sql('Facilities', conn, if_exists='replace', index = False)
# Create test query to make sure the data was loaded correctly
rows = cur.fetchall()
for row in rows:
    print(row)
# Close the connection (when done with ntbk uncomment this line)
conn.close()
# Print the first 5 rows of each table
print(df_book.head())
print(df_mem.head())
print(df_fac.head())

df_book.dtypes
# For Q13, we need to convert the starttime column to a datetime object
df_book['starttime'] = pd.to_datetime(df_book['starttime'])
df_book.dtypes


   bookid  facid  memid       starttime  slots
0       0      3      1  7/3/2012 11:00      2
1       1      4      1   7/3/2012 8:00      2
2       2      6      0  7/3/2012 18:00      2
3       3      7      1  7/3/2012 19:00      2
4       4      8      1  7/3/2012 10:00      1
   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone  recommendedby        joindate  
0  (000) 000-0000            NaN   7/1/2012 0:00  
1    555-555-5555            NaN  7/2/2012 12:02  
2    555-555-5555            NaN  7/2/2012 12:08  
3  (844) 693-0723            NaN   7/3/2012 9:32  
4  (833) 942-4710            1.0  7/

bookid                int64
facid                 int64
memid                 int64
starttime    datetime64[ns]
slots                 int64
dtype: object

In [2]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!
cur.execute('SELECT name AS facility_name, total_revenue FROM (SELECT name, SUM(	CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost * slots ELSE Facilities.membercost * slots END) as total_revenue	FROM Bookings JOIN Members ON Bookings.memid = Members.memid JOIN Facilities ON Bookings.facid = Facilities.facid GROUP BY name) as Facilities_Revenue WHERE total_revenue < 1000 ORDER BY total_revenue DESC;')
column_names = [description[0] for description in cur.description]
rows = cur.fetchall()
query1 = pd.DataFrame(rows, columns=column_names)
print(query1)


   facility_name  total_revenue
0     Pool Table          270.0
1  Snooker Table          240.0
2   Table Tennis          180.0


In [3]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order.
cur.execute('SELECT m1.surname AS member_lastname, m1.firstname AS member_firstname, m2.surname AS rec_lastname, m2.firstname AS rec_firstname FROM Members AS m1 LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid WHERE m1.recommendedby != 0 ORDER BY m1.surname, m1.firstname;');
column_names = [description[0] for description in cur.description]
rows = cur.fetchall()
query_df2 = pd.DataFrame(rows, columns=column_names)
print(query_df2)

      member_lastname member_firstname rec_lastname rec_firstname
0               Bader         Florence     Stibbons        Ponder
1               Baker             Anne     Stibbons        Ponder
2               Baker          Timothy      Farrell        Jemima
3              Boothe              Tim       Rownam           Tim
4             Butters           Gerald        Smith        Darren
5              Coplin             Joan        Baker       Timothy
6             Crumpet            Erica        Smith         Tracy
7                Dare            Nancy     Joplette        Janice
8             Genting          Matthew      Butters        Gerald
9                Hunt             John      Purview     Millicent
10              Jones            David     Joplette        Janice
11              Jones          Douglas        Jones         David
12           Joplette           Janice        Smith        Darren
13          Mackenzie             Anna        Smith        Darren
14        

In [4]:
# Q12: Find the facilities with their usage by member, but not guests.
cur.execute('SELECT name AS facility_name, COUNT(memid) AS member_usage FROM Bookings JOIN Facilities ON Bookings.facid = Facilities.facid WHERE memid != 0 GROUP BY name ORDER BY member_usage DESC;')
column_names = [description[0] for description in cur.description]
rows = cur.fetchall()
query_df3 = pd.DataFrame(rows, columns=column_names)
print(query_df3)

     facility_name  member_usage
0       Pool Table           783
1    Snooker Table           421
2   Massage Room 1           421
3     Table Tennis           385
4  Badminton Court           344
5   Tennis Court 1           308
6   Tennis Court 2           276
7     Squash Court           195
8   Massage Room 2            27


In [6]:
# Q13: Find the facilities usage by month, but not guests.
cur.execute("SELECT f.name AS facility, SUM(CASE WHEN strftime('%m', b.starttime) = '01' THEN 1 ELSE 0 END) AS January, SUM(CASE WHEN strftime('%m', b.starttime) = '02' THEN 1 ELSE 0 END) AS February, SUM(CASE WHEN strftime('%m', b.starttime) = '03' THEN 1 ELSE 0 END) AS March, SUM(CASE WHEN strftime('%m', b.starttime) = '04' THEN 1 ELSE 0 END) AS April, SUM(CASE WHEN strftime('%m', b.starttime) = '05' THEN 1 ELSE 0 END) AS May, SUM(CASE WHEN strftime('%m', b.starttime) = '06' THEN 1 ELSE 0 END) AS June, SUM(CASE WHEN strftime('%m', b.starttime) = '07' THEN 1 ELSE 0 END) AS July, SUM(CASE WHEN strftime('%m', b.starttime) = '08' THEN 1 ELSE 0 END) AS August, SUM(CASE WHEN strftime('%m', b.starttime) = '09' THEN 1 ELSE 0 END) AS September, SUM(CASE WHEN strftime('%m', b.starttime) = '10' THEN 1 ELSE 0 END) AS October, SUM(CASE WHEN strftime('%m', b.starttime) = '11' THEN 1 ELSE 0 END) AS November, SUM(CASE WHEN strftime('%m', b.starttime) = '12' THEN 1 ELSE 0 END) AS December FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid INNER JOIN Members AS m ON b.memid = m.memid WHERE m.firstname NOT LIKE 'GUEST' AND m.surname NOT LIKE 'GUEST' GROUP BY f.name ORDER BY facility");
column_names = [description[0] for description in cur.description]
rows = cur.fetchall()
query_df5 = pd.DataFrame(rows, columns=column_names)
print(query_df5)

          facility  January  February  March  April  May  June  July  August  \
0  Badminton Court        0         0      0      0    0     0     0       0   
1   Massage Room 1        0         0      0      0    0     0     0       0   
2   Massage Room 2        0         0      0      0    0     0     0       0   
3       Pool Table        0         0      0      0    0     0     0       0   
4    Snooker Table        0         0      0      0    0     0     0       0   
5     Squash Court        0         0      0      0    0     0     0       0   
6     Table Tennis        0         0      0      0    0     0     0       0   
7   Tennis Court 1        0         0      0      0    0     0     0       0   
8   Tennis Court 2        0         0      0      0    0     0     0       0   

   September  October  November  December  
0          0        0         0         0  
1          0        0         0         0  
2          0        0         0         0  
3          0        0  